In [1]:
%pip install ruprompts 

Note: you may need to restart the kernel to use updated packages.


# ruPROMPTs tutorial

This [tutorial](https://github.com/ai-forever/ru-prompts/blob/main/notebooks/detox-russe-train-python.ipynb) presents an exaple of prompt-tuning with ruPROMTS framework for the detoxification task.

In [2]:
!wget -O task2/train.csv https://raw.githubusercontent.com/skoltech-nlp/russe_detox_2022/main/data/input/train.tsv
!wget -O task2/dev.csv https://raw.githubusercontent.com/skoltech-nlp/russe_detox_2022/main/data/input/dev.tsv

--2023-04-09 11:18:16--  https://raw.githubusercontent.com/skoltech-nlp/russe_detox_2022/main/data/input/train.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1902888 (1.8M) [text/plain]
Saving to: ‘task2/train.csv’

task2/train.csv     100%[===================>]   1.81M  8.50MB/s    in 0.2s    

2023-04-09 11:18:17 (8.50 MB/s) - ‘task2/train.csv’ saved [1902888/1902888]

--2023-04-09 11:18:17--  https://raw.githubusercontent.com/skoltech-nlp/russe_detox_2022/main/data/input/dev.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Leng

In [2]:
import pandas as pd

df = pd.read_csv("task2/train.tsv", sep="\t")
df.drop(["index"], axis=1, inplace=True)
df.to_csv("task2/train.tsv", index=False, sep="\t")

KeyError: "['index'] not found in axis"

## Training

In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

Load the dataset:

In [5]:
!pip install datasets

In [4]:
from datasets import load_dataset

datasets = load_dataset("csv", data_files={"train": "task2/train.tsv", "validation": "task2/dev.tsv"}, sep="\t")
train_dataset = datasets["train"]
valid_dataset = datasets["validation"]

Found cached dataset csv (/home/jovyan/.cache/huggingface/datasets/csv/default-6f14a3b9c6758b34/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/2 [00:00<?, ?it/s]

Load the backbone:

In [5]:
from transformers import GPT2LMHeadModel, AutoTokenizer

backbone_id = "sberbank-ai/rugpt3large_based_on_gpt2"

model = GPT2LMHeadModel.from_pretrained(backbone_id)
tokenizer = AutoTokenizer.from_pretrained(backbone_id, pad_token="<pad>", eos_token="<pad>")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Define the prompt format:

In [6]:
from ruprompts import PromptFormat

prompt_format = PromptFormat("<P*100>{toxic_comment}<P*20>")

Define the parametrization of trainable embeddings:

In [7]:
from ruprompts import TensorPromptProvider
from transformers import set_seed

set_seed(1)

prompt_provider = TensorPromptProvider()

Compose prompt format and prompt provider into prompt object and apply it to the model and tokenizer, i.e. add special tokens to the tokenizer and modify the layer of input embeddings of the model:

In [8]:
from ruprompts import Prompt

prompt = Prompt(prompt_format, prompt_provider)
prompt.patch(model, tokenizer)

Preprocess the data:
1. format the data entries with the specified prompt format
2. tokenize the resulting sequences
3. truncate the `truncation_field` if sequence length exceeds `max_tokens`

In [9]:
from ruprompts import Text2TextPreprocessor

preprocessor = Text2TextPreprocessor(
    prompt_format=prompt_format,
    tokenizer=tokenizer,
    target_field="neutral_comment1",
    max_tokens=1792,
    truncation_field="toxic_comment",
)

train_dataset = train_dataset.map(preprocessor)
valid_dataset = valid_dataset.map(preprocessor)

Loading cached processed dataset at /home/jovyan/.cache/huggingface/datasets/csv/default-6f14a3b9c6758b34/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-2f6b0322d76cd568.arrow
Loading cached processed dataset at /home/jovyan/.cache/huggingface/datasets/csv/default-6f14a3b9c6758b34/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317/cache-0b69788a14bad05d.arrow


Define training arguments:

In [10]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir=".",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=1,
    eval_steps=1000,
    save_steps=1000,
    logging_steps=100,
    evaluation_strategy="steps",
    save_strategy="steps",
    logging_strategy="steps",
    save_total_limit=2,
    metric_for_best_model="eval_loss",
    learning_rate=0.1,
    max_steps=100000,
    report_to="tensorboard",
    # report_to=["tensorboard", "wandb"],  # uncomment to log to WandB
    logging_dir="logs",
    seed=1,
)

Choose optimization options:

In [11]:
from transformers.optimization import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(prompt_provider.parameters(), lr=training_args.learning_rate)
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=2000,
    num_training_steps=training_args.max_steps,
)

/home/jovyan/.imgenv-jupyter-server-dfcbc5cc9913-0/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Define the callbacks and start training:

In [ ]:
from transformers import Trainer
from ruprompts.callbacks import (
    FreezeTransformerUnfreezePrompt,
    ReduceCheckpoint,
    SavePretrainedPrompt,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    data_collator=preprocessor.collate_fn(),
    optimizers=(optimizer, scheduler),
    callbacks=[FreezeTransformerUnfreezePrompt(), ReduceCheckpoint(), SavePretrainedPrompt(prompt)],
)

trainer.train()

max_steps is given, it will override any value given in num_train_epochs
The following columns in the training set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: neutral_comment1, toxic_comment, neutral_comment2, neutral_comment3. If neutral_comment1, toxic_comment, neutral_comment2, neutral_comment3 are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 6948
  Num Epochs = 29
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 2
  Gradient Accumulation steps = 1
  Total optimization steps = 100000
  Number of trainable parameters = 760484352
You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
1000,2.201700,1.911890
2000,1.692000,1.783948
3000,1.598500,1.709484
4000,1.633900,1.644775
5000,1.541400,1.598003
6000,1.516300,1.548342
7000,1.483300,1.600356
8000,1.574000,1.540647


The following columns in the evaluation set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: neutral_comment1, toxic_comment, neutral_comment2, neutral_comment3. If neutral_comment1, toxic_comment, neutral_comment2, neutral_comment3 are not expected by `GPT2LMHeadModel.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 800
  Batch size = 2
Saving model checkpoint to ./checkpoint-1000
Configuration saved in ./checkpoint-1000/config.json
Configuration saved in ./checkpoint-1000/generation_config.json
Model weights saved in ./checkpoint-1000/pytorch_model.bin
Deleting older checkpoint [checkpoint-3000] due to args.save_total_limit
The following columns in the evaluation set don't have a corresponding argument in `GPT2LMHeadModel.forward` and have been ignored: neutral_comment1, toxic_comment, neutral_comment2, neutral_comment3. If neutral_comment1, toxic_comment, neutral_comment2, neutral_comment3 are not e

In [ ]:
!nvidia-smi

## Inference

Load prompt from the last checkpoint:

In [19]:
from transformers import pipeline

prompt = Prompt.from_pretrained(f"./checkpoint-{training_args.max_steps}")

ppln = pipeline("text2text-generation-with-prompt", prompt=prompt, model=model, tokenizer=tokenizer, device=0)

In [20]:
ppln({"toxic_comment": "Ублюдок, мать твою, а ну иди сюда"}, do_sample=False)

[{'generated_text': 'А ну иди сюда'}]

Run inference:

In [25]:
from tqdm import tqdm
import transformers

transformers.logging.set_verbosity_error()

beam_count = 10

predictions = []
    
for i in tqdm(valid_dataset["toxic_comment"]):
    options = ppln(
        {"toxic_comment": i},
        do_sample=False,
        num_beams=beam_count,
        num_return_sequences=beam_count,
    )

    options = [i["generated_text"].replace("<pad>", "") for i in options]
    print(i)
    for opt in options:
        print('-', opt)
    answer = sorted(options, key=len)[-1]  # get longest answer
    print('--->', answer)
    predictions.append(answer)

with open("task2/subm.txt", "w") as f:
    f.writelines(list(map(lambda x: x.replace("\n", " ") + "\n", predictions)))

  0%|          | 1/800 [00:00<04:23,  3.03it/s]

пиздеж! температуры горения хватит чтобы её расплавить к херам..
- Температуры горения хватит, чтобы её расплавить
- Температуры горения хватит чтобы её расплавить
- Температуры горения хватит, чтобы расплавить
- Температуры горения хватит, чтобы ее расплавить
- температуры горения хватит чтобы её расплавить
- Температура горения хватит, чтобы её расплавить
- Температура горения хватит чтобы её расплавить
- Температуры горения хватит чтобы ее расплавить
- Температуры горения хватит чтоб её расплавить
- Температуры горения хватит чтобы расплавить
---> Температура горения хватит, чтобы её расплавить


  0%|          | 2/800 [00:00<04:00,  3.32it/s]

а ты чмо там был.ты вообще служил.гандон
- ты вообще служил
- А ты там был
- а ты там был.ты вообще служил
- Ты вообще служил
- Ты там был
- а ты там был
- А ты там был. Ты вообще служил
- Там ты был
- а ты там был. ты вообще служил
- А ты там был.
---> а ты там был. ты вообще служил


  0%|          | 3/800 [00:00<03:42,  3.59it/s]

...

долбаеб, решил ресонуться, купил бы кольцо, цветы и на колено, вот что нужно кате а не пафос с вертолета
- Решил ресонуться, купил бы кольцо, цветы и на колено, вот что нужно кате а не пафос с вертолета
- решил ресонуться, купил бы кольцо, цветы и на колено, вот что нужно кате а не пафос с вертолета
- решил ресонуться, купил бы кольцо, цветы и на колено
- Решил ресонуться, купил бы кольцо, цветы и на колено, вот что нужно кате
- Решил ресонуться, купил бы кольцо, цветы и на колено
- Решил ресонуться, купил бы кольцо, цветы и на колено, вот что нужно а не пафос с вертолета
- Решил ресонуться, купил бы кольцо, цветы и на колено, вот что нужно кате а не пафос с вертолёта
- Решил ресонуться, купил бы кольцо, цветы и на колени
- Решил ресонуться, купил бы кольцо, цветы и на колени, вот что нужно кате а не пафос с вертолета
- решил ресонуться, купил бы кольцо, цветы и на колено, вот что нужно кате
---> Решил ресонуться, купил бы кольцо, цветы и на колени, вот что нужно кате а не пафос с верт

100%|█████████▉| 799/800 [05:00<00:00,  2.26it/s]

такому уроду пасти баранов страшно доверить а он в думе сидит.
- такому человеку пасти баранов страшно доверить а он в думе сидит
- такому доверять нельзя а он в думе сидит
- такому человеку пасти баранов страшно доверить а он в думе сидит.
- такому доверить нельзя а он в думе сидит
- такому человеку пасти овец страшно доверить а он в думе сидит
- Такому человеку доверять нельзя
- такому человеку доверять нельзя а он в думе сидит
- такому человеку доверить пасти баранов страшно доверить а он в думе сидит
- такому доверять нельзя а он в думе сидит.
- такому доверять нельзя, а он в думе сидит
---> такому человеку доверить пасти баранов страшно доверить а он в думе сидит


100%|██████████| 800/800 [05:00<00:00,  2.66it/s]

самий пидараз здесь бартамео он уйдёть всё будеть нармалньно я в барсу верю
- самый плохой человек здесь бартамео он уйдёть всё будеть нармалньно я в барсу верю
- самый плохой человек здесь бартамео он уйдёт всё будеть нармалньно я в барсу верю
- Самый плохой человек здесь бартамео он уйдёт всё будеть нармалньно я в барсу верю
- Самый плохой человек здесь бартамео он уйдёть всё будеть нармалньно я в барсу верю
- самый плохой здесь бартамео он уйдёть всё будеть нармалньно я в барсу верю
- самый нехороший здесь бартамео он уйдёть всё будеть нармалньно я в барсу верю
- самый нехороший человек здесь бартамео он уйдёть всё будеть нармалньно я в барсу верю
- Самый плохой человек здесь бартамео он уйдёт всё будеть нормально я в барсу верю
- самый плохой человек тут бартамео он уйдёть всё будеть нармалньно я в барсу верю
- самый плохой здесь бартамео он уйдёт всё будеть нармалньно я в барсу верю
---> самый нехороший человек здесь бартамео он уйдёть всё будеть нармалньно я в барсу верю
